In [21]:
import pandas as pd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import os
import numpy as np
import seaborn as sns
from matplotlib.colors import ListedColormap

In [22]:
# Load statistical summeries per database
drugbank_avg_tar_df = pd.read_csv('results/avg_targ_distance_analysis/avg_targ_distance_tox_drugbank.csv')
ddinter_avg_tar_df = pd.read_csv('results/avg_targ_distance_analysis/avg_targ_distance_tox_ddinter.csv')
drugbank_avg_tar_df["Database"] = "DrugBank"
ddinter_avg_tar_df["Database"] = "DDInter"

drugbank_smiles_df = pd.read_csv('results/smiles/mfp_tox_tanimoto_drugbank.csv')
ddinter_smiles_df = pd.read_csv('results/smiles/mfp_tox_tanimoto_ddinter.csv')
drugbank_smiles_df["Database"] = "DrugBank"
ddinter_smiles_df["Database"] = "DDInter"

drugbank_twohop_df = pd.read_csv('results/two_hop_analysis/two_hop_targ_jaccard_drugbank.csv')
ddinter_twohop_df = pd.read_csv('results/two_hop_analysis/two_hop_targ_jaccard_ddinter.csv')
drugbank_twohop_df["Database"] = "DrugBank"
ddinter_twohop_df["Database"] = "DDInter"

# Combine dataframes
combined_corr_df = pd.concat([drugbank_avg_tar_df, ddinter_avg_tar_df,
                                 drugbank_smiles_df, ddinter_smiles_df,
                                 drugbank_twohop_df, ddinter_twohop_df], ignore_index=True)

In [23]:
combined_corr_df

,Toxicity Metric,Test_Type,Comparison,Test_Statistic,P_value,P_value_Corrected,Effect_Size_Type,Effect_Size_Value,Database
0,Average Target Distance,Kruskal-Wallis,All Groups,H=6.6724e+03,0.000000e+00,NaN,Eta-Squared H,0.106488,DrugBank
1,Average Target Distance,Dunn Post-Hoc,Major vs Minor,Dunn Z,NaN,0.000000e+00,Cliff's Delta,-0.559198,DrugBank
2,Average Target Distance,Dunn Post-Hoc,Major vs Moderate,Dunn Z,NaN,0.000000e+00,Cliff's Delta,-0.264475,DrugBank
3,Average Target Distance,Dunn Post-Hoc,Moderate vs Minor,Dunn Z,NaN,1.621376e-274,Cliff's Delta,-0.275129,DrugBank
4,Average Target Distance,Jonckheere-Terpstra,Increasing Trend (Minor->Major),Z=-7.8777e+01,1.000000e+00,NaN,r,-0.314704,DrugBank
5,Average Target Distance,Jonckheere-Terpstra,Decreasing Trend (Major->Minor),Z=7.8777e+01,0.000000e+00,NaN,r,0.314704,DrugBank
6,Average Target Distance,Kruskal-Wallis,All Groups,H=4.6187e+02,5.085574e-101,NaN,Eta-Squared H,0.015895,DDInter
7,Average Target Distance,Dunn Post-Hoc,Major vs Minor,Dunn Z,NaN,8.716835e-59,Cliff's Delta,0.311921,DDInter
8,Average Target Distance,Dunn Post-Hoc,Major vs Moderate,Dunn Z,NaN,6.903661e-76,Cliff's Delta,0.157342,DDInter
9,Average Target Distance,Dunn Post-Hoc,Moderate vs Minor,Dunn Z,NaN,3.231599e-20,Cliff's Delta,0.197053,DDInter


In [25]:
metrics = [
    'Average Target Distance', 
    'Morgan Fingerprint Tanimoto Similarity', 
    '2-Hop Neighboring Proteins Jaccard'
    ]

databases = ['DrugBank', 'DDInter'] 

# Retrieve all necessary data for one test/database/metric combination 
def get_stats_data(df, metric, db, test_type, comparison=''):
    """
    Filters the DataFrame for a specific test result and extracts the Statistic, 
    P-value, Effect Size, and Effect Symbol.
    """
    
    # Determine which P-value column to use and the Effect Size Symbol
    if test_type == 'Kruskal-Wallis':
        p_col = 'P_value'
        e_symbol = '\\eta^2'
    elif test_type == 'Dunn Post-Hoc':
        p_col = 'P_value_Corrected'
        e_symbol = '\\delta'
    elif test_type == 'Jonckheere-Terpstra':
        p_col = 'P_value'
        e_symbol = 'r'
    else:
        return np.nan, np.nan, np.nan, ''

    # Filter the DataFrame
    print(df[(df['Toxicity Metric'] == metric) &
        (df['Database'] == db)].head(3))
    result_row = df[
        (df['Toxicity Metric'] == metric) &
        (df['Database'] == db) &
        (df['Test_Type'] == test_type) &
        (df['Comparison'] == comparison)
    ]
    print(f"result_now: {result_row}")
    if result_row.empty:
        # Return standard NaN placeholders if no data found
        return np.nan, np.nan, np.nan, e_symbol
    
    # Extract values
    row = result_row.iloc[0]
    
    # Test Statistic (assumed to be H for KW, Z for JT, and NaN/blank for Dunn)
    stat = row.get('Test_Statistic', np.nan)
    
    p = row.get(p_col, np.nan)
    e = row.get('Effect_Size_Value', np.nan)

    return stat, p, e, e_symbol


# Function to format the P-value and Effect Size for LaTeX
def format_p_effect(p_value, effect_size, effect_symbol):
    """Formats P-value and Effect Size into the string P-value (Effect Size Symbol = Value)."""
    
    # Check for missing data
    if pd.isna(p_value) or pd.isna(effect_size):
         print("Missing data for P-value or Effect Size.")
         return '-'
         
    # P-value Formatting
    if p_value >= 0.05:
        # For non-significant P-values: e.g., 0.10, >0.99. All must be in math mode.
        p_str = "${:.3f}$".format(p_value)
    else:
        # Handle P-value of zero or near-zero
        if p_value <= 1e-300: # Use a safe threshold
            p_str = "${:.3f}$".format(p_value)
        else:
            # For significant P-values: use scientific notation. All must be in math mode.
            exponent = int(np.floor(np.log10(p_value)))
            mantissa = p_value / 10**exponent
            p_str = "${:.3f} \\times 10^{{{:d}}}$".format(mantissa, exponent)
    
    # Effect Size Formatting
    # Ensure correct sign formatting for the effect size
    sign = "+" if effect_size >= 0 else ""
    effect_str = "{}{:.3f}".format(sign, effect_size)
    
    # Combine P-value and effect size with the correct symbol. All symbols and values must be in math mode.
    return f"{p_str} ($\\mathbf{{{effect_symbol}}}={effect_str}$)"

# --- Function to format the Test Statistic ---
def format_statistic(stat):
    """Formats the Test Statistic (H or Z) into a clean LaTeX string."""
    print(f"Formatting statistic: {stat} (type: {type(stat)})")
    
    stat_type = str(stat.split('=')[0])
    stat_val = float(stat.split('=')[1])
    print(stat_val)

    abs_stat = abs(stat_val)
    if abs_stat >= 10:
        exponent = int(np.floor(np.log10(abs_stat)))
        mantissa = abs_stat / 10**exponent
        sign = "-" if stat_val < 0 else ""
        print(f"stat type: {stat_type}, stat value: {stat_val}")
        return f"${sign}{mantissa:.3f} \\times 10^{{{exponent:d}}}$"
    else:
        # Fixed decimal for smaller stats, enclosed in math mode
        return "${:.3f}$".format(stat_val)

# Generate LaTex Table
def generate_latex_table_from_df(df, metrics_list, databases_list):
    """Generates the final LaTeX code string for the summary table based on the DataFrame."""
    
    # Define the 6 Test rows for each Synergy Score, matching the DataFrame columns
    test_rows_map = [
        ('Kruskal-Wallis', 'Kruskal-Wallis', 'All Groups'),
        ('Dunn: Major/Minor', 'Dunn Post-Hoc', 'Major vs Minor'),
        ('Dunn: Major/Moderate', 'Dunn Post-Hoc', 'Major vs Moderate'),
        ('Dunn: Moderate/Minor', 'Dunn Post-Hoc', 'Moderate vs Minor'),
        ('Jonckheere-Terpstra: Increasing Toxicity', 'Jonckheere-Terpstra', 'Increasing Trend (Minor->Major)'),
        ('Jonckheere-Terpstra: Decreasing Toxicity', 'Jonckheere-Terpstra', 'Decreasing Trend (Major->Minor)'),
    ]
    
    # Start of LaTeX Table Code (Using your requested structure)
    latex_code = [
        "\\begin{table}[htbp]",
        "\\centering",
        "\\small", # Requested formatting
        "\\caption{\\textbf{Toxicity scoring metrics compared to toxicity categories.} Table containing the results from statistical assessment of whether common principles used in toxicity scores trend with known DDI categories. The first section tests the similarity of drug structure in a combination by calculating the Tanimoto Similarity of the Morgan Fingerprint representations of both drugs in a combination. The second section examines whether closer average target distance in a drug combination is associated with toxicity levels. Finally, the last section calculates the overlap (Jaccard Similarity) of each drug target's neighborhood within two hops of the protein target on the STRING PPIN. The results for the Kruskal-Wallis, Dunn with Bonferroni correction, and Jonckheere-Terpstra tests are included for both the DrugBank and DDInter datasets. The Jonckheere-Terpestra Increasing Toxicity assesses whether the distribution of the toxicity scoring metric increases when toxicity categories become more severe, while the Jonckheere-Terpstra Decreasing Toxicity test evaluates if the toxicity scoring metric distributions increase when the toxicity categories decrease in severity.}",
        "\\label{tab:s_toxscore}",
        "\\vspace{0.5em}", # Requested formatting
        "\\resizebox{\\textwidth}{!}{", 
        "\\scriptsize", # Use smallest common font size
        "\\begin{tabular}{ll|rr|rr}", # l=Score, l=Test, r r = DrugBank, r r = DDInter
        "\\toprule",
    ]
    
    # Header Rows
    header_row_1 = ["\\textbf{Overlap Metric}", "\\textbf{Test}", 
                    "\\multicolumn{2}{c|}{\\textbf{DrugBank}}", 
                    "\\multicolumn{2}{c}{\\textbf{DDInter}}"]
    latex_code.append(" & ".join(header_row_1) + " \\\\")
    
    # P-value column now includes Effect Size
    header_row_2 = ["\\textbf{Score}", "", "\\textbf{Test Statistic}", 
                    "\\textbf{P-value (Effect Size)}", 
                    "\\textbf{Test Statistic}", 
                    "\\textbf{P-value (Effect Size)}"]
    latex_code.append(" & ".join(header_row_2) + " \\\\")

    latex_code.append("\\midrule")
    
    # Data Rows 
    for i, metric in enumerate(metrics_list):
        # Escape underscores in synergy scores (e.g., S\_max) for text mode
        escaped_metric = metric.replace('_', '\\_') 
        score_cell = f"\\multirow{{{len(test_rows_map)}}}{{*}}{{\\textbf{{{escaped_metric}}}}}"
        
        for j, (display_name, test_type, comparison_col) in enumerate(test_rows_map):
            row_data = {}
            is_statistic_required = test_type == 'Kruskal-Wallis' or test_type == 'Jonckheere-Terpstra'

            for db in databases_list: # Loop over ['DrugBank', 'DDInter']
                # 1. Get Data
                stat, p, e, e_symbol = get_stats_data(df, metric, db, test_type, comparison_col)
                print(f"Retrived for {db}, {metric}, {test_type}, {comparison_col}: stat={stat}, p={p}, e={e}, e_symbol={e_symbol}")
                # 2. Format Strings
                stat_str = format_statistic(stat) if is_statistic_required else '-'
                p_e_str = format_p_effect(p, e, e_symbol)

                row_data[db] = {
                    'stat': stat_str,
                    'p_e': p_e_str
                }
            # 4. Construct Row
            row_cells = [
                score_cell if j == 0 else "", 
                display_name,
                row_data['DrugBank']['stat'],
                row_data['DrugBank']['p_e'],
                row_data['DDInter']['stat'],
                row_data['DDInter']['p_e']
            ]
            latex_code.append(" & ".join(row_cells) + " \\\\")

        # Add horizontal rule after each metric group, except the last one
        if i < len(metrics_list) - 1:
            latex_code.append("\\midrule")
            
    # --- Footer ---
    latex_code.append("\\bottomrule")
    latex_code.append("\\end{tabular}")
    latex_code.append("}") # Closing bracket for \resizebox
    latex_code.append("\\end{table}")
    
    return "\n".join(latex_code)

# Now generate the table
latex_output = generate_latex_table_from_df(combined_corr_df, metrics, databases)

# Print the output to the console (standard behavior)
print(latex_output)

filename = "results/tox_scoring_metrics.tex"
with open(filename, 'w', encoding='utf-8') as f:
            f.write(latex_output)
print(f"Success! LaTeX table saved to {filename}")
print(latex_output)


           Toxicity Metric       Test_Type         Comparison Test_Statistic  \
0  Average Target Distance  Kruskal-Wallis         All Groups   H=6.6724e+03   
1  Average Target Distance   Dunn Post-Hoc     Major vs Minor         Dunn Z   
2  Average Target Distance   Dunn Post-Hoc  Major vs Moderate         Dunn Z   

   P_value  P_value_Corrected Effect_Size_Type  Effect_Size_Value  Database  
0      0.0                NaN    Eta-Squared H           0.106488  DrugBank  
1      NaN                0.0    Cliff's Delta          -0.559198  DrugBank  
2      NaN                0.0    Cliff's Delta          -0.264475  DrugBank  
result_now:            Toxicity Metric       Test_Type  Comparison Test_Statistic  \
0  Average Target Distance  Kruskal-Wallis  All Groups   H=6.6724e+03   

   P_value  P_value_Corrected Effect_Size_Type  Effect_Size_Value  Database  
0      0.0                NaN    Eta-Squared H           0.106488  DrugBank  
Retrived for DrugBank, Average Target Distance, Krus